# Update Dates and Deadlines in Gradescope

First, we need to load the user's gradescope email and password. We are reading these from the environment and support a `.env` file in the local directory with these variables.

To use this notebook, you need to set the env vars `GRADESCOPE_EMAIL` and `GRADESCOPE_PASSWORD` to their respective values.

In [ ]:
import os
import dotenv
dotenv.load_dotenv()

assert "GRADESCOPE_EMAIL" in os.environ, "Be sure the environment variable GRADESCOPE_EMAIL is set to your e-mail address."
GRADESCOPE_EMAIL = os.environ["GRADESCOPE_EMAIL"]
assert "GRADESCOPE_PASSWORD" in os.environ, "Be sure the environment variable GRADESCOPE_EMAIL is set to your password."
GRADESCOPE_PASSWORD = os.environ["GRADESCOPE_PASSWORD"]
assert "GRADESCOPE_COURSE" in os.environ, "Be sure the environment variable GRADESCOPE_COURSE is set to the course ID number from URL."
GRADESCOPE_COURSE = os.environ["GRADESCOPE_COURSE"]

 Automatically reload our helper module.

In [ ]:
# The following snippet will automatically reload imported code in helpers.
%load_ext autoreload
%autoreload 2

Next, we are importing the necessary modules and establishing a chrome webdriver.

In [ ]:
from gradescope_helpers import GradescopeDriver, GradescopeCourse, GradescopeAssignment
gradescope = GradescopeDriver(GRADESCOPE_EMAIL, GRADESCOPE_PASSWORD)

Get a handle on the course we're working with.

In [ ]:
course = gradescope.get_course(GRADESCOPE_COURSE)
print(course)

Now let's get a list of all the course's assignments.

In [ ]:
assignments = course.get_assignments()
assignments

In [ ]:
%run Schedule.ipynb

In [ ]:
from helpers import Kind
from datetime import timedelta
LATE_PERIODS: dict[Kind, timedelta] = {
    Kind.LS: timedelta(1),
    Kind.CQ: timedelta(1),
    Kind.EX: timedelta(2),
    Kind.RD: timedelta(2)
}

from datetime import datetime, time
for day in syllabus:
    plans = syllabus[day]
    for plan in plans:
        if plan.kind == Kind.LS or plan.kind == Kind.CQ or plan.kind == Kind.EX or plan.kind == Kind.RD:
            kind = str(plan.kind)[-2:]
            lesson = f"{kind}{plan.n:02}"
            if lesson not in assignments: continue
            gs_lesson = assignments[lesson]
            gs_lesson.load()
            dirty = False

            release_date = datetime.combine(plan.date - timedelta(days=2), time(0, 0, 0))
            if release_date != gs_lesson.release_date:
                gs_lesson.release_date = release_date
                dirty = True
            
            due_date = datetime.combine(plan.deadline, time(23, 59, 00))
            if due_date != gs_lesson.due_date:
                gs_lesson.due_date = due_date
                dirty = True
    
            if not gs_lesson.allow_late_submissions:
                gs_lesson.allow_late_submissions = True
                dirty = True

            late_due_date = datetime.combine(plan.deadline + LATE_PERIODS[plan.kind], time(23, 59, 00))
            if late_due_date != gs_lesson.late_due_date:
                gs_lesson.late_due_date = late_due_date
                dirty = True

            if dirty:
                print("Updating", lesson)
                gs_lesson.persist()